# Notebook 1: Preprocessing 

The data that we will be using to build the win probability model will be the wyscout open figshare data. 
The actual event data for an entire season is in a single large JSON file. We will use pandas to load in the data, process it into a format we want and build some basic features in this notebook. In the next notebook, we will continue to build the features and then finally train a model in the last notebook. 

-------------------------

According to the model explained by ASA, the features to build are as follows: 

#### Game state: 
 * Score differential, 
 * player differential, 
 * number of goals scored, 
 * number of yellow and red cards issued

#### Team attribute: 
 * Home team 
 * expansion season indicators

#### Team strength: 
 * Offensive and defensive measures of strength for each team and its opponent; calculated using goals added (g+) for and against over a set of recent games

#### Chance creation: 
 * Difference in offensive g+ earned in recent minutes

### Target: 
 Have they scored a goal from that moment on (binary variable)

### Example dataframe:

|Score differential   | Player differential   | Number of goals scored   |Number of yellow cards   |Number of red cards   |  Home team | Strength difference  | g+ difference |
|---|---|---|---|---|---|---|---|
|1   |0   |1   |1   |0   |1   |1.2   |0.9|
|   |   |   |   |   |   |   |   |
|   |   |   |   |   |   |   |   |

In the first row, the home team is

* winning by 1 goal (score differential) 
* no red cards (player differential = 0; red cards = 0)
* have scored one goal
* have one yellow card
* are the home team (home team = 1)
* strength difference is a 1.2 xT. The blog used g+ difference over a set of recent games, say 4 matches. I do not have access to g+ so I will use xT here. 
* difference in g+ over the last n minutes, say 10 minutes. Again, we will use xT here. 

### Some other considerations:

* The chronological order of the matches matters. This is because of the team strength features to indicate recent form. 
* Using xT in place of g+ to calculate the pre-match team strength indicators as well as the running game flow.

In [1]:
import json
from glob import glob
import os

import pandas as pd; pd.set_option("display.max_columns", None)
from pandas import json_normalize
import numpy as np
from tqdm import tqdm

In [2]:
WYSCOUT_DATA_FOLDER = "../wyscout_figshare_data" ## path where the wyscout data is extracted
PROCESSED_DATA_FOLDER = "../processed-data" ## where you want to save the processed csv files
EVENTS_FILE_NAME = os.path.join(WYSCOUT_DATA_FOLDER, "events\events_England.json") ## the league which you want to train the model on (Premier League here)
MATCHES_FILE_NAME = os.path.join(WYSCOUT_DATA_FOLDER, "matches\matches_England.json")

##events file
with open(EVENTS_FILE_NAME) as f:
    events = json.load(f)
df = json_normalize(events)
df['tags_list'] = df["tags"].apply(lambda x: [d['id'] for d in x])

##matches file
with open(MATCHES_FILE_NAME) as f:
    matches = json.load(f)
matches_dict = {match['wyId']:match for match in matches} 

##xt file - download from https://karun.in/blog/data/open_xt_12x8_v1.json
with open("../expected_threat.json") as f:
    xtd = np.array(json.load(f))

In [3]:
df.head()

,eventId,subEventName,tags,playerId,positions,matchId,eventName,teamId,matchPeriod,eventSec,subEventId,id,tags_list
0,8,Simple pass,[{'id': 1801}],25413,"[{'y': 49, 'x': 49}, {'y': 78, 'x': 31}]",2499719,Pass,1609,1H,2.758649,85,177959171,[1801]
1,8,High pass,[{'id': 1801}],370224,"[{'y': 78, 'x': 31}, {'y': 75, 'x': 51}]",2499719,Pass,1609,1H,4.946850,83,177959172,[1801]
2,8,Head pass,[{'id': 1801}],3319,"[{'y': 75, 'x': 51}, {'y': 71, 'x': 35}]",2499719,Pass,1609,1H,6.542188,82,177959173,[1801]
3,8,Head pass,[{'id': 1801}],120339,"[{'y': 71, 'x': 35}, {'y': 95, 'x': 41}]",2499719,Pass,1609,1H,8.143395,82,177959174,[1801]
4,8,Simple pass,[{'id': 1801}],167145,"[{'y': 95, 'x': 41}, {'y': 88, 'x': 72}]",2499719,Pass,1609,1H,10.302366,85,177959175,[1801]


In [4]:
def get_goals(vals, side):
    
    tags, event_name, team_id = vals
    if side == 'home':
        return (101 in tags and team_id == home_team_id) or (102 in tags and team_id == away_team_id) ## 101 is a goal, 102 is an own goal
    elif side == 'away':
        return (101 in tags and team_id == away_team_id) or (102 in tags and team_id == home_team_id)
    
def get_xt_value(vals):
    x1,x2,y1,y2 = vals
    return xtd[y2][x2] - xtd[y1][x1]        

match_ids = sorted(df['matchId'].unique())

for match_id in tqdm(match_ids):
    
    match_df = df.query("matchId == @match_id").copy()
    
    match_md = matches_dict[match_id] ##match meta data

    home_team_id, = [int(key) for key in match_md['teamsData'] if match_md['teamsData'][key]['side'] == 'home']
    away_team_id, = [int(key) for key in match_md['teamsData'] if match_md['teamsData'][key]['side'] == 'away']
    
    ## assign columns
    match_df["home_goals"] = 0
    match_df["away_goals"] = 0
    
    match_df['home_number_of_yellows'] = 0
    match_df['away_number_of_yellows'] = 0

    match_df['home_number_of_players'] = 11
    match_df['away_number_of_players'] = 11
    
    ## get goals
    shots_df = match_df.query("eventName == 'Shot'")
    
    home_goal_idxs = shots_df.loc[shots_df[['tags_list', 'eventName', 'teamId']].apply(get_goals, side='home', axis=1)].index
    away_goal_idxs = shots_df.loc[shots_df[['tags_list', 'eventName', 'teamId']].apply(get_goals, side='away', axis=1)].index
    
    for idx in home_goal_idxs:
        match_df.loc[idx:, "home_goals"] +=1
    for idx in away_goal_idxs:
        match_df.loc[idx:, "away_goals"] +=1
    
    ## get yellow cards
    home_first_yellow_idxs = match_df[match_df[['tags_list', 'teamId']].apply(lambda vals: 1702 in vals[0] and vals[1] == home_team_id, axis=1)].index
    away_first_yellow_idxs = match_df[match_df[['tags_list', 'teamId']].apply(lambda vals: 1702 in vals[0] and vals[1] == away_team_id, axis=1)].index
    
    for idx in home_first_yellow_idxs:
        match_df.loc[idx:, "home_number_of_yellows"] +=1
    for idx in away_first_yellow_idxs:
        match_df.loc[idx:, "away_number_of_yellows"] +=1
    
    ## get red cards
    home_red_idxs = match_df[match_df[['tags_list', 'teamId']].apply(lambda vals: (1701 in vals[0] or 1703 in vals[0]) and vals[1] == home_team_id, axis=1)].index
    away_red_idxs = match_df[match_df[['tags_list', 'teamId']].apply(lambda vals: (1701 in vals[0] or 1703 in vals[0]) and vals[1] == away_team_id, axis=1)].index

    for idx in home_red_idxs:
        match_df.loc[idx:, "home_number_of_players"] -=1
    for idx in away_red_idxs:
        match_df.loc[idx:, "away_number_of_players"] -=1
    
    
    ## get pass xt values
    pass_df = match_df.loc[(match_df['eventName'] == 'Pass') & (match_df['tags_list'].astype(str).str.contains("1801"))].copy()
    pass_df[['x1', 'y1', 'x2', 'y2']] = pd.DataFrame(pass_df['positions'].\
                                apply(lambda data: [data[0]['x'], data[0]['y'], data[1]['x'], data[1]['y']]).\
                                tolist(), index=pass_df.index)

    pass_df['x1_idx'] = np.clip(np.int64(pass_df['x1'].values//(100/12)), a_min=0, a_max=11)
    pass_df['x2_idx'] = np.clip(np.int64(pass_df['x2'].values//(100/12)), a_min=0, a_max=11)
    
    pass_df['y1_idx'] = np.clip(np.int64(pass_df['y1'].values//(100/8)), a_min=0, a_max=7)
    pass_df['y2_idx'] = np.clip(np.int64(pass_df['y2'].values//(100/8)), a_min=0, a_max=7)
    pass_df['xt_value'] = pass_df[['x1_idx','x2_idx','y1_idx','y2_idx']].apply(get_xt_value, axis=1)

    match_df.loc[pass_df.index, "xt_value"] = pass_df['xt_value']
    
    ##save 
    filename = os.path.join(PROCESSED_DATA_FOLDER, f"{match_id}_{home_team_id}_{away_team_id}.csv")
    match_df.to_csv(filename, index=False)    

100%|███████████████████████████████████| 380/380 [01:18<00:00,  4.83it/s]
